In [2]:
#!/usr/bin/env python3

import pickle
import sys
import numpy as np
import matplotlib.pyplot as plt
from operator import add

from dataTy import dataTy
from dataTy import Output
from dataTy import ResultHelper

def open_pickle(default_name):
    pfile = default_name
    # Use this in terminal py
    #if len(sys.argv) > 1:
        # first arg is filename
    #    pfile = sys.argv[1]
    print("Opening " + pfile)
    with open(pfile,'rb') as f:
        result = pickle.load(f)
    return result

class Summarizer:
    def __init__(self, result):
        print("{0:<15}".format(""), end='')
        for config in ResultHelper.getConfigs(result):
            print("{0:<15} ".format(config), end='')
        print("")
        for proj in ResultHelper.getProjs(result):
            print("{0:<15}".format(proj), end= '')
            for config in ResultHelper.getConfigs(result):
                avg_time = round(ResultHelper.getErrorOrAvgTime(result[config][proj]), 3)
                print("{0:<15} ".format(avg_time), end='')
            print("")
                
class Printer:
    def __init__(self, result, the_config = None, the_proj = None):
        if the_config == None :
             configs = result.keys()
        else:
            configs = [the_config]
        for config in configs:
            output_of_proj = result[config]
            if the_proj == None:
                projs = output_of_proj.keys()
            else:
                projs = [the_proj]
                    
            for proj in projs:
                output = output_of_proj[proj]
                print("{0}-{1}, {2}".format(config, proj, round(ResultHelper.getErrorOrAvgTime(output),3)))
                print("Profiling Times: " + str(output.prof_time))
                for name in output.prof_data:
                    d = output.prof_data[name]
                    print("  {0:<20}| {1:<10}| {2}".format(name, d.Count, round(d.Value,3)))

class config:
    width = 1
    margin = 1
    left_margin = 1
    def rename_config(config):
        renametable = {"omp-offload" : "clang", "omp-offload-1d" : "1D", "omp-offload-bulk": "bulk", "omp-offload-at": "bulk+at"}
        ret = renametable.get(config)
        if ret == None:
            print(config + "is not found in renametable")
            return config
        else:
            return ret
    save_img = True

class PlotPrinter:
    def submit(name):
        plt.tight_layout()
        if config.save_img:
            plt.savefig("plot/" + name + ".png", format='png',dpi=300, edgecolor='k')
        else:
            plt.show()
        plt.clf()

class ColumnChartPrinter:
    def plot(result):
        # Get projs
        projs = ResultHelper.getProjs(result)
        configs = ResultHelper.getConfigs(result)
        proj_count = len(projs)
        config_count = len(configs)
        if config_count == 0:
            return
        if proj_count == 0:
            return
        #config.width = 1/(1+config_count)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count + 1), config_count + 1)
        last_col = xpos_base[-1]

        for c in configs:
            # prepare data
            height = []
            xpos = [x + config.width * n for x in xpos_base]
            for p in projs:
                time = ResultHelper.getAvg(result[c][p].times)
                height.append(time)
            plt.bar(xpos, height, width=config.width, label=c)
            last_col = xpos[-1]
            n += 1
        plt.ylabel('Execution Time(sec)', fontweight='bold')
        plt.xlabel('Benchmarks', fontweight='bold')
        plt.title('Overview', fontweight='bold')
        # Insert in the center
        xticks = [x + config_count / 2 * config.width for x in xpos_base]
        plt.xticks(xticks, projs)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)
        plt.xlim(xpos_base[0] - config.left_margin, last_col + config.margin)
        PlotPrinter.submit("all")

class StackChartPrinter:
    def plotStackChar(result, configs, proj, ind, metrics):
        N = len(configs)
        btn = [0] * N
        plist = []
        for m in metrics:
            time = []
            for c in configs:
                time.append(float(result[c][proj].prof_data[m].Value))
            p = plt.bar(ind, time, config.width, bottom=btn)
            plist.append(p)
            # Increase btn
            btn = list(map(add, btn, time))
        return plist

    def plot(result, proj):
        configs = ResultHelper.getConfigs(result)
        #metrics = ["Other", "Runtime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        metrics = ["Runtime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        config_count = len(configs)
        ind = np.arange(0, config_count * (config.width + 1),  config.width + 1)
        #ind = np.arange(N)    # the x locations for the groups

        plist = StackChartPrinter.plotStackChar(result, configs, proj, ind, metrics)

        plt.ylabel('Execution Time(sec)', fontweight='bold')
        plt.xlabel('Configs', fontweight='bold')
        plt.title('[Breakdown] ' + proj, fontweight='bold')
        plt.xticks(ind, [config.rename_config(x) for x in configs]) #plt.yticks(np.arange(0, 81, 10))
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=3)
        plt.xlim(ind[0]- config.left_margin, ind[-1] + config.margin)
        PlotPrinter.submit(proj)

class OffloadVS1DNormPrinter:
    # Like stackChart
    def plot(result):
        projs = ResultHelper.getProjs(result)
        configs = ["omp-offload", "omp-offload-1d"]
        metrics = ["Runtime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        for c in configs:
            if result.get(c) == None:
                print(c + " key not found")
                return
        # Gen xpos and xticks
        xpos = []
        xticks = []
        ind = np.arange(0, 2 * (config.width + 0.5), config.width + 0.5).tolist() # the x locations for the groups
        for proj in projs:
            xpos = xpos + ind
            xticks += [proj, proj+"1d"]
            ind = [x + 2 * config.width + 2.5 * config.width for x in ind]

        btn = [0] * len(projs) * 2
        plist = []
        # calculate the factor
        factors = {}
        for proj in projs:
            sum = 0
            for m in metrics:
                sum += float(result["omp-offload"][proj].prof_data[m].Value)
            factors[proj] = 100/sum
                
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                for c in configs:
                    time.append(f*float(result[c][proj].prof_data[m].Value))
                    
            plist.append(plt.bar(xpos, time, config.width, bottom=btn))
            btn = list(map(add, btn, time))
        plt.ylabel('Execution Time(%)', fontweight='bold')
        #plt.xlabel('Configs', fontweight='bold')
        plt.title("Direct Offload vs 1D Refactor Comparison", fontweight='bold')
        # shift because of rotation
        plt.xticks([x - config.width/2 for x in xpos], xticks, rotation=40)
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=3)
        plt.xlim(xpos[0] - config.left_margin, xpos[-1] + config.margin)
        PlotPrinter.submit("Offloadvs1DCompare")

class ATOptPrinter:
    def plot(results):
        for r in results:
            first = results[r]
            break
        opts = results.keys()
        
        projs = ResultHelper.getProjs(first)
        the_config = "omp-offload-at"
        #metrics = [ "Kernel"]
        metrics = ["Runtime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        
        # Gen xpos and xticks
        xpos = []
        xticks = []
        ind = np.arange(0, 2 * (config.width + 0.5), config.width + 0.5).tolist() # the x locations for the groups
        for proj in projs:
            xpos = xpos + ind
            xticks += [proj, proj+"-opt"]
            ind = [x + 2 * config.width + 2.5 * config.width for x in ind]

        btn = [0] * len(projs) * 2
        plist = []
        # calculate the factor
        factors = {}
        for proj in projs:
            sum = 0
            for m in metrics:
                sum += float(first[the_config][proj].prof_data[m].Value)
            factors[proj] = 100/sum
                
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                for o in opts:
                    time.append(f*float(results[o][the_config][proj].prof_data[m].Value))
                    
            plist.append(plt.bar(xpos, time, config.width, bottom=btn))
            btn = list(map(add, btn, time))
        plt.ylabel('Execution Time(%)', fontweight='bold')
        #plt.xlabel('Configs', fontweight='bold')
        plt.title("AT Opt Comparison", fontweight='bold')
        # shift because of rotation
        plt.xticks([x - config.width/2 for x in xpos], xticks, rotation=40)
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=3)
        plt.xlim(xpos[0] - config.left_margin, xpos[-1] + config.margin)
        plt.ylim(top=110)
        PlotPrinter.submit("ATOptEffect")
        
# Compare different at optimization
def at_opt():
    files = {}
    files["O0"]   = "./results/at-O0.p"
    files["Osm"]  = "./results/at-Osm.p"
    
    results = {}
    for f in files:
        results[f] = pickle.load(open(files[f], "rb"))
        print("Open "+ files[f])
        ResultHelper.preprocessing(results[f])
    ATOptPrinter.plot(results)
    
def main():
    # Read from pickle
    result = open_pickle("./results/result.p")
    ResultHelper.preprocessing(result)
    Summarizer(result)
    Printer(result)
    #Printer(result, the_proj="pathfinder")
    #Printer(result, the_proj="kmeans", the_config="omp-offload")
    #Printer(result, the_proj="kmeans", the_config="omp-offload-bulk")
    return
    
    
    Printer(result, the_proj="kmeans", the_config="omp-offload")
    Printer(result, the_proj="kmeans", the_config="omp-offload-bulk")
    
    StackChartPrinter.plot(result, "kmeans")
    return 
    #StackChartPrinter.plot(result, "backprop")
    OffloadVS1DNormPrinter.plot(result) 
    ColumnChartPrinter.plot(result)
    projs = ResultHelper.getProjs(result)
    for p in projs:
        StackChartPrinter.plot(result, p)


#config.save_img = False
plt.style.use('seaborn-deep') # choose style https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html

if __name__ == "__main__":
    main()
    #at_opt()
    

Opening ./results/result.p
               omp-offload     omp-dce         
backprop       7.745           6.675           
pathfinder     0.285           0.27            
omp-offload-backprop, 7.745
Profiling Times: 7.495
  Runtime             | 196674    | 1.572
  Kernel              | 4         | 0.047
  H2DTransfer         | 393338    | 1.142
  UpdatePtr           | 393352    | 2.3
  D2HTransfer         | 131114    | 1.798
  RTTarget            | 4         | 0.047
  RTDataBegin         | 131112    | 4.903
  RTDataUpdate        | 0         | 0.0
  RTDataEnd           | 65562     | 1.888
  Parallelism         | 4         | 4352.0
  ATTableSize         | 0         | 0.0
  TargetMem           | 0         | 0.0
  Other               | 1         | 0.637
omp-offload-pathfinder, 0.285
Profiling Times: 0.275
  Runtime             | 201       | 0.003
  Kernel              | 99        | 0.002
  H2DTransfer         | 103       | 0.005
  UpdatePtr           | 500       | 0.006
  D2HTransfer     